In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import movie_reviews
import random
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import re

In [ ]:
# Load data from GitHub
url = "https://raw.githubusercontent.com/geoffswc/UCSF-DSOS-Python-SQL-Workshops/refs/heads/main/data/movie_reviews.csv"
df = pd.read_csv(url)

# Rename columns so they match your notebook’s expected names
df = df.rename(columns={'text': 'review', 'review': 'label'})

# Shuffle rows (to preserve same behavior as before)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Inspect
print(df.head())


In [ ]:
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['label'], test_size=0.2, random_state=42
)

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')  # for WordNet data
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_analyzer(text):
    # Lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Split words
    words = text.split()
    # Lemmatize each word
    words = [word for word in words if word not in stop_words]
    return [lemmatizer.lemmatize(word) for word in words]

In [ ]:
vectorizer = CountVectorizer(
    analyzer=lemmatize_analyzer,
    stop_words='english',
    max_features=5000
)


In [ ]:
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=41))
])


In [ ]:
vectorizer = pipeline.named_steps['vectorizer']
classifier = pipeline.named_steps['classifier']

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
importances = classifier.feature_importances_

df_features = pd.DataFrame({
    'word': feature_names,
    'importance': importances
})

In [ ]:
df_features

In [ ]:
X_train_vec = vectorizer.transform(X_train)
X_dense = X_train_vec.toarray()

# Binary labels for convenience
y_train_bin = (y_train == 'pos').astype(int)

# Count per class
count_positive = X_dense[y_train_bin==1].sum(axis=0)
count_negative = X_dense[y_train_bin==0].sum(axis=0)

# Total count across all documents
total_count = X_dense.sum(axis=0)
feature_names = vectorizer.get_feature_names_out()
importances = classifier.feature_importances_

df_features = pd.DataFrame({
    'word': feature_names,
    'importance': importances,
    'count_positive': count_positive,
    'count_negative': count_negative,
    'total_count': total_count
})

# Sort by importance
df_features_sorted = df_features.sort_values(by='importance', ascending=False)



In [ ]:
df_features_sorted.head(20)

In [ ]:
df_features_sorted.tail(20)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

y_pred = pipeline.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")

# Precision and Recall for the 'pos' class
prec = precision_score(y_test, y_pred, pos_label=1)
rec = recall_score(y_test, y_pred, pos_label=1)
print(f"Precision (pos): {prec:.2f}")
print(f"Recall (pos): {rec:.2f}")

# Full classification report for all classes
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


In [ ]:
# 1. Predict labels and probabilities
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

# 2. Extract probabilities for 'pos' and 'neg' explicitly
# Make sure to match the correct column for each class
class_order = pipeline.classes_  # e.g., ['neg', 'pos']
prob_pos = y_prob[:, list(class_order).index(1)]
prob_neg = y_prob[:, list(class_order).index(0)]

# 3. Build the DataFrame
df_reviews = pd.DataFrame({
    'text': X_test,
    'actual': y_test,
    'predicted': y_pred,
    'prob_pos': prob_pos,
    'prob_neg': prob_neg
})

# 4. Display the first few rows
df_reviews.head()

In [ ]:
!pip install pandasql

In [ ]:
import pandasql
pysqldf = lambda q: pandasql.sqldf(q, globals())

# Make sure df_reviews exists and has the columns: text, actual, predicted, prob_pos, prob_neg

# SQL query:
# 1. Filter where actual != predicted
# 2. Compute the maximum of prob_pos and prob_neg as the model's confidence
# 3. Sort descending by that confidence
# 4. Limit to top 10

query = """
SELECT
    text,
    actual,
    predicted,
    prob_pos,
    prob_neg,
    CASE
        WHEN prob_pos > prob_neg THEN prob_pos
        ELSE prob_neg
    END AS pred_confidence,
    CASE
        WHEN actual = 'pos' AND predicted != 'pos' THEN 'false_negative'
        WHEN actual != 'pos' AND predicted = 'pos' THEN 'false_positive'
        ELSE 'other'
    END AS error_type
FROM df_reviews
WHERE actual != predicted
ORDER BY pred_confidence DESC
LIMIT 10;
"""

top10_worst_sql = pysqldf(query)



In [ ]:
# Display the result
top10_worst_sql